## Testing the model

### Import Libraries

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.feature import hog
from joblib import load,dump
import os
from tqdm import tqdm,trange

### Parameters: Part 1

These must match those of the model

In [2]:
patchsize_yx = (128,128)
orientations=8
pixels_per_cell=(16, 16)
cells_per_block=(2, 2)

These can be changed to improve sliding window/non-max suppression

In [3]:
sws_percent = [22,25]
overlap_thresh = 0.6

### Functions for Running The Model


Sliding Window

In [4]:
def sliding_window(image, stepSize, windowSize,returnSize = patchsize_yx):# image is the input, step size is the no.of pixels needed to skip and windowSize is the size of the actual window
    # slide a window across the image
    for y1 in range(0, image.shape[1] - windowSize[1], stepSize):# this line and the line below actually defines the sliding part and loops over the x and y coordinates
        for x1 in range(0, image.shape[0] - windowSize[0], stepSize):
            # yield the current window
            #yield (x, y, image[y: y + windowSize[1], x:x + windowSize[0]])
            y2 = y1 + windowSize[0]
            x2 = x1 + windowSize[1]
            patch = image[y1: y2, x1: x2]
            if patch.shape[0:2] == windowSize:
                return_patch = cv2.resize(patch,returnSize)
                yield (x1, y1,x2, y2, return_patch)

Area funcs for non-max suppression

In [5]:
def area(rect):
    x1,y1,x2,y2 = rect
    return (x2-x1+1)*(y2-y1+1)

def area_intersect(rect1,rect2):
    x1,y1,x2,y2 = rect1
    x3,y3,x4,y4 = rect2
    if x2 < x3:
        return 0
    if x4 < x1:
        return 0
    if y2 < y3:
        return 0
    if y4 < y1:
        return 0
    X1 = max(x1,x3)
    X2 = min(x2,x4)
    Y1 = max(y1,y3)
    Y2 = min(y2,y4)
    return area((X1,Y1,X2,Y2))

def area_union(rect1,rect2):
    return area(rect1) + area(rect2) - area_intersect(rect1,rect2)

def union_to_intersect(rect1,rect2):
    return area_intersect(rect1,rect2)/area_union(rect1,rect2)

The Big Boy for counting Among Us

In [6]:
def how_many_amogus(model,test_img,ret_labeled_img = False ,sws_percent= sws_percent, overlap_thresh= overlap_thresh):
    sws_array = np.array(sws_percent)
    imh,imw,nchannel = test_img.shape
    short_side = min(imh,imw)
    sliding_window_sizes = np.round(sws_array*short_side/100).astype(int)
    step_size = np.ceil(sliding_window_sizes/8).astype(int)


    patches = []
    for i in range(len(sws_percent)):
        tup = (sliding_window_sizes[i],sliding_window_sizes[i])
        patches += list(sliding_window(test_img,step_size[i],tup))

    patches_hog = np.array([hog(patch, orientations=orientations, pixels_per_cell=pixels_per_cell,cells_per_block=cells_per_block, visualize=False, channel_axis=-1)\
        for _,_,_,_,patch in patches])

    labels = model.predict(patches_hog)
    confidence_in_class1 = model.predict_proba(patches_hog)

    labels_confidence = []
    for i,patch in enumerate(patches):
        if labels[i] == 1:
            x1,y1,x2,y2,_ = patch
            labels_confidence.append((confidence_in_class1[i,1],x1,y1,x2,y2))

    labels_confidence.sort(reverse=True)
    true_amogus = []
    for element in labels_confidence:
        conf = element[0]
        rect = element[1:]
        not_duplicate = True
        for _,true_rect in true_amogus:

            if union_to_intersect(rect,true_rect) > overlap_thresh:
                not_duplicate = False
                break
        if not_duplicate:
            true_amogus.append((conf,rect))

    if ret_labeled_img:
        image = test_img.copy()
        for conf,rect in true_amogus:
            x1,y1,x2,y2, = rect
            image = cv2.rectangle(image,(x1,y1),(x2,y2),(255,0,0),4)
            
        for conf,rect in true_amogus:
            x1,y1,x2,y2, = rect
            image = cv2.putText(image,str(round(conf,5)),(x1,y1),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,192,192),2,cv2.LINE_AA)
        return len(true_amogus),image
    else:
        return len(true_amogus)

For evalution

In [7]:
def MAE(y_true,y_pred):
     y_true = np.array(y_true)
     y_pred = np.array(y_pred)
     return np.mean(np.abs(y_true - y_pred))

def RMSE(y_true,y_pred):
     y_true = np.array(y_true)
     y_pred = np.array(y_pred)
     return np.sqrt(np.mean((y_true - y_pred)**2))

### Parameters: Part 2

In [8]:
test_img_dir = "Test_Pic/set3"
answer_file = 'count.txt'
model_path = 'Model/model.joblib'
numofpic = 1505

## Main Code

### Load Model and Initialize some to-be-used lists 

In [9]:
model = load(model_path)
result = []
answers = []

### Loop for image

In [ ]:
#for i in trange(numofpic):
for i in trange(20):
    test_img = cv2.imread(f'{test_img_dir}/img{i}.jpg')
    test_img = cv2.cvtColor(test_img,cv2.COLOR_BGR2RGB)
    x = how_many_amogus(model,test_img)
    result.append(x)

### Loop for true values

In [ ]:
with open(f'{test_img_dir}/{answer_file}','r') as file:
    for line in file:
        i, a = line.split()
        answers.append(int(a))

### MAE

In [ ]:
print(MAE(result,answers))

### RMSE

print(RMSE(result,answers))